# Prediction As Classification
Continuing the 2800-HK price prediction from classification perspective

## Step 1: Import Modules and load raw data

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import keras

Using TensorFlow backend.


In [2]:
#Import sklearn
from sklearn.preprocessing import scale

In [3]:
#Import Keras module
from keras import optimizers
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
#pretty plots
%matplotlib inline

In [5]:
#Import price data
#Load the historical prices of 2800-HK, with lags 1 to lag 20
price_hist_data = pd.read_csv('price_only.csv', skiprows=1, parse_dates=['Date']).set_index(['Date'])
price_hist_data.head()

,P,P_L1,P_L2,P_L3,P_L4,P_L5,P_L6,P_L7,P_L8,P_L9,...,P_L11,P_L12,P_L13,P_L14,P_L15,P_L16,P_L17,P_L18,P_L19,P_L20
Date,,,,,,,,,,,,,,,,,,,,,
2007-08-31,24.35,23.80,23.35,23.70,23.90,23.30,23.35,22.70,22.05,21.85,...,20.95,21.70,22.35,22.30,22.05,22.70,22.85,22.15,22.25,22.85
2007-09-03,24.30,24.35,23.80,23.35,23.70,23.90,23.30,23.35,22.70,22.05,...,20.80,20.95,21.70,22.35,22.30,22.05,22.70,22.85,22.15,22.25
2007-09-04,24.30,24.30,24.35,23.80,23.35,23.70,23.90,23.30,23.35,22.70,...,21.85,20.80,20.95,21.70,22.35,22.30,22.05,22.70,22.85,22.15
2007-09-05,24.35,24.30,24.30,24.35,23.80,23.35,23.70,23.90,23.30,23.35,...,22.05,21.85,20.80,20.95,21.70,22.35,22.30,22.05,22.70,22.85
2007-09-06,24.50,24.35,24.30,24.30,24.35,23.80,23.35,23.70,23.90,23.30,...,22.70,22.05,21.85,20.80,20.95,21.70,22.35,22.30,22.05,22.70


In [6]:
#Import Fundamentals Data
fund_data = pd.read_csv('new_index_data.csv', skiprows=1, parse_dates=['Date']).set_index(['Date'])
fund_data.head() 

,Close,High,Low,Ask,Bid,20D Vol,MA5,MA15,MA12,MA20,...,DY_LTM,DY_NTM,ADV_VOL,PAYOUT,ANALYST_SENTIMENT,EPS_GRW_FY1,EPS_GRW_FY2,PE_NTM,PE_LTM,C2D_LTM
Date,,,,,,,,,,,,,,,,,,,,,
2007-08-31,24.35,24.35,24.35,24.35,24.35,2.367823,23.82,22.841667,22.696667,22.6225,...,2.782797,2.880128,99.419898,48.051962,1.909071,30.724490,2.717280,16.695058,16.975805,58.050474
2007-09-03,24.30,24.30,24.30,24.30,24.30,2.266743,23.90,23.120832,22.830000,22.7250,...,2.784563,2.892308,11.570023,48.305678,1.835359,31.005439,1.876752,16.709085,16.925571,58.154835
2007-09-04,24.30,24.30,24.30,24.30,24.30,2.259649,24.02,23.412500,22.960000,22.8325,...,2.784339,2.894377,69.555725,48.307102,1.725886,31.040968,1.891823,16.697662,16.919413,58.177640
2007-09-05,24.35,24.35,24.35,24.35,24.35,2.172140,24.22,23.620832,23.136667,22.9075,...,2.767348,2.867314,24.265623,48.267351,1.741908,30.716929,1.878484,16.841225,17.017692,58.195446
2007-09-06,24.50,24.50,24.50,24.50,24.50,2.160638,24.36,23.825000,23.373333,22.9975,...,2.775339,2.878880,88.845002,48.288397,1.696151,31.336520,1.861943,16.780660,16.975485,58.224743


In [7]:
#Import Global index data
idx_data = pd.read_csv('indices.csv', skiprows=1, parse_dates=['Date']).set_index(['Date'])
idx_data.head()

,Hang Seng Index,SSE Composite Index,ASX All Ordinaries,India S&P BSE SENSEX,TOPIX,KOSPI Composite Index,Taiwan TAIEX,FTSE Bursa Malaysia KLCI,FTSE Straits Times Index,Philippines PSE PSEi,...,Turkey BIST 100,S&P 500,DJ Industrial Average,Colombia IGBC,Canada S&P/TSX Composite,Brazil Bovespa Index,Mexico IPC,Israel TA-125,Saudi Arabia All Share (TASI),FTSE JSE All Share
Date,,,,,,,,,,,,,,,,,,,,,
2007-08-31,23984.14,5218.825,6248.3,15318.60,1608.25,1873.24,8982.16,1273.93,3328.43,3365.29,...,50198.60,1473.99,13357.74,10728.74,13660.48,54637.24,30347.86,1034.67,8226.97,28660.35
2007-09-03,23904.09,5321.055,6272.5,15422.05,1605.44,1881.81,8979.96,1284.14,3321.36,3369.14,...,49936.94,1473.99,13357.74,10750.79,13660.48,54832.51,30797.60,1047.33,8017.54,28887.48
2007-09-04,23886.07,5294.045,6297.1,15465.40,1596.74,1874.74,8922.98,1283.75,3308.81,3312.30,...,50032.59,1489.42,13448.86,10880.85,13755.23,55250.47,30932.71,1054.69,7878.70,29051.96
2007-09-05,24069.17,5310.716,6274.3,15446.15,1569.47,1865.59,8913.85,1297.93,3375.02,3342.35,...,49421.38,1472.29,13305.47,10819.91,13683.28,54407.83,30809.55,1048.70,7853.66,28696.67
2007-09-06,24050.40,5393.660,6265.3,15616.31,1568.52,1888.81,9017.08,1298.85,3399.49,3326.53,...,49601.39,1478.55,13363.35,10844.40,13795.69,54569.00,30816.95,1033.23,7853.66,28850.19


## Step 2: Pre-process Raw Data
1. Generate labels
2. Apply lags to global index data
3. Normalize features

In [8]:
#Generate Labels from Price History Data
#Generate UP/DOWN labels from log change
cutoff_perc = 0.0005 #0.05% return as cuttoff to define UP label
lag = 1 #forward returns

labels = np.zeros([price_hist_data.shape[0]])

#Caluclate log-returns
ret = np.log(price_hist_data['P'].shift(-lag)/price_hist_data['P'])
labels[ret > cutoff_perc] = 1

In [9]:
#Applying lags to index data
#Seperate the indices into 2 classes - lag or no_lag
no_lag = [0, 1, 2, 4, 5, 6, 9, 10]
lag = [i for i in range(0,idx_data.shape[1]) if i not in no_lag]

#Processing the dataset by applying appropriate lags
lagged_data = idx_data.iloc[:,lag].shift(1)
idx_data = pd.concat([idx_data.iloc[:,no_lag], lagged_data], axis=1)


In [10]:
#Remove first row
idx_data = idx_data.iloc[1:, :]
price_hist_data = price_hist_data.iloc[1:, :]
fund_data = fund_data.iloc[1:, :]
labels=labels[1:]

In [11]:
#Check Dimensions to make sure everythings right before continuing..
print("Shape of idx_data: ", idx_data.shape)
print("Shape of price_hist_data: ", price_hist_data.shape)
print("Shape of fund_data: ", fund_data.shape)
print("Shape of labels: ", labels.shape)

Shape of idx_data:  (2483, 42)
Shape of price_hist_data:  (2483, 21)
Shape of fund_data:  (2483, 30)
Shape of labels:  (2483,)


In [12]:
#Training data
X = np.array(pd.concat([price_hist_data, idx_data, fund_data], axis=1))
y = to_categorical(labels, num_classes=2)

In [13]:
#Checking whether there are NAs
[np.sum(np.isnan(X), axis=0) > 0] == True

False

In [14]:
len(y.shape)


2

## Step 3: Split training, validation and test set

In this stage we have a look-ahead bias free set of data (X) and the labels y. Next, we will need to:
- Normalize the input data. To avoid look ahead bias, we will z-score the features, using ONLY the training set.
- Next, generate input data into LSTM network. We will need an overlapping sequence at 1-day window as input samples. Specifically suppose the *timestep* is 240, we will have a list of array consists of *number of rows of X* - 240 entries, each element has dimentions (240, num_of_features)


In [15]:
#Function to split raw data into training, validation and test set, returns a numpy array.
def split_data(input_data=[], train_size=0.8, val_size=0.2, test_size=0):
    
    #------------------------------------------------
    #PARAM: input_data: numpy nd array
    #PARAM: training_size: size of training set in decimal
    #PARAM: val_size: size of validation set in decimal
    #PARAM: test_size: size of test set in decimal
    #OUTPUT: tuple (train_set, validation_set, test_set)
    #------------------------------------------------
    
    #First check whether traing_size + val_size + test_size = 1 and each of the entries are positive
    assert(train_size + val_size + test_size==1), "Sum of training, validation and test size needs to be 1!"
    assert(train_size * val_size * test_size > 0), "Sizes have to be positive!"
    
    #Check input_data type is numpy array, after casting
    if type(input_data) != 'numpy.ndarray':
        input_data = np.array(input_data) 
    
    assert(isinstance(input_data, np.ndarray)), "Input has to be a numpy array!"
    
    
    #Calculate cut-off points
    train_cut_index = int(train_size * input_data.shape[0])
    val_cut_index = train_cut_index + int(val_size * input_data.shape[0])
    
    #Split the data
    if len(input_data.shape) == 1:
        train, val, test = input_data[:train_cut_index], input_data[train_cut_index:val_cut_index], input_data[val_cut_index:]
    else:
        train, val, test = input_data[:train_cut_index,:], input_data[train_cut_index:val_cut_index, :], input_data[val_cut_index:, :]
    
    return train, val, test

In [16]:
#------------
#TEST OUTPUT
#------------
#Split data
X_train, X_val, X_test = split_data(X, train_size=0.7, val_size=0.15, test_size=0.15)
y_train, y_val, y_test = split_data(y, train_size=0.7, val_size=0.15, test_size=0.15)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1738, 93)
(372, 93)
(373, 93)
(1738, 2)
(372, 2)
(373, 2)


In [17]:
#Function to re-structure the data to get batches. Re-shape the data to have overlapping training set for time-series learning.
def get_inputs(input_data, timesteps):
    
    #First get the total number of samples generated
    n_samples = input_data.shape[0] - timesteps
    
    #Initiate empty list to store the sequence
    output = []
    
    #Generate the sequences
    for ii in range(0, n_samples+1):
        if len(input_data.shape) == 1:
            output.append(scale(input_data[ii:ii+timesteps]))
        else:
            output.append(scale(input_data[ii:ii+timesteps,:]))
        
    return output

In [18]:
#------------
#TEST OUTPUT
#------------
#get inputs
t = np.reshape(np.arange(1,51), (10,5))
test = get_inputs(t, 5)

#print shapes
print('Length of output: ',len(test))
#print('Tensor shape of each training set', test[0].shape)
test

Length of output:  6


C:\Users\User\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int32 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


[array([[-1.41421356, -1.41421356, -1.41421356, -1.41421356, -1.41421356],
        [-0.70710678, -0.70710678, -0.70710678, -0.70710678, -0.70710678],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.70710678,  0.70710678,  0.70710678,  0.70710678,  0.70710678],
        [ 1.41421356,  1.41421356,  1.41421356,  1.41421356,  1.41421356]]),
 array([[-1.41421356, -1.41421356, -1.41421356, -1.41421356, -1.41421356],
        [-0.70710678, -0.70710678, -0.70710678, -0.70710678, -0.70710678],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.70710678,  0.70710678,  0.70710678,  0.70710678,  0.70710678],
        [ 1.41421356,  1.41421356,  1.41421356,  1.41421356,  1.41421356]]),
 array([[-1.41421356, -1.41421356, -1.41421356, -1.41421356, -1.41421356],
        [-0.70710678, -0.70710678, -0.70710678, -0.70710678, -0.70710678],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.70710678,

In [19]:
timesteps = 20

#Normalize training data
X_train_np = get_inputs(X_train, timesteps)
X_val_np = get_inputs(X_val, timesteps)
X_test_np = get_inputs(X_test, timesteps)

y_train_np = get_inputs(y_train, timesteps)
y_val_np = get_inputs(y_val, timesteps)
y_test_np = get_inputs(y_test, timesteps)


In [20]:
print(len(X_train_np))
print(len(y_train_np))
print(len(X_val_np))
print(len(y_val_np))

1719
1719
353
353


In [21]:
print(X_train_np[0].shape)
print(y_train_np[0].shape)

(20, 93)
(20, 2)


In [36]:
new_train = np.reshape(X_train_np, (len(X_train_np), timesteps, 93))
new_target = np.reshape(y_train_np, (len(y_train_np), timesteps, 2))
new_val_X = np.reshape(X_val_np, (len(X_val_np), timesteps, 93))
new_val_y = np.reshape(y_val_np, (len(y_val_np), timesteps, 2))

In [24]:
#Define Parameters
learning_rate = 0.001
epochs= 100
loss = 'binary_crossentropy'
dropout=0.5

In [25]:
#Optimizer
optimizer = optimizers.Adam(lr=learning_rate)

In [40]:
#Model Architect
#-----------------

model = Sequential()

#Stack LSTM Cells
model.add(LSTM(50, input_shape=(timesteps, 93), batch_size=1, return_sequences=True, stateful=True))

model.add(LSTM(25, batch_size=1, return_sequences=True, stateful=True))

#model.add(Dense(64, activation='relu'))

#model.add(Dense(32, activation='relu'))

model.add(Dense(2, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (1, 20, 50)               28800     
_________________________________________________________________
lstm_9 (LSTM)                (1, 20, 25)               7600      
_________________________________________________________________
dense_5 (Dense)              (1, 20, 2)                52        
Total params: 36,452
Trainable params: 36,452
Non-trainable params: 0
_________________________________________________________________


In [41]:
#Compile
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [42]:
#Fit
fitted = model.fit(new_train, new_target, epochs=epochs, validation_data=(new_val_X, new_val_y), batch_size=1, verbose=1, shuffle=False)

Train on 1719 samples, validate on 353 samples
Epoch 1/100
1719/1719 [==============================] - 144s - loss: 0.4058 - acc: 0.0630 - val_loss: 0.5194 - val_acc: 0.0708
Epoch 2/100
1719/1719 [==============================] - 142s - loss: 0.2542 - acc: 0.0642 - val_loss: 0.5196 - val_acc: 0.0693
Epoch 3/100
  86/1719 [>.............................] - ETA: 126s - loss: 0.4780 - acc: 0.0564

KeyboardInterrupt: 

# Below this line are old codes

In [ ]:
#Split into training, val and test set
#Parameters
train_size = 0.9
val_size = ( 1- train_size) * 0.5
timestep = 5

In [ ]:
#Calculate cutoff index values
train_cut_index = int(train_size * X.shape[0] - train_size * X.shape[0] % timestep)
val_cut_index = train_cut_index + int(val_size * X.shape[0] - val_size * X.shape[0] % timestep)
last_test_index = int(X.shape[0] - X.shape[0] % timestep)
print(train_cut_index)
print(val_cut_index)
print(last_test_index)

In [ ]:
#Spit Training, Validation and Test
X_train = np.array(X[0:train_cut_index,:])
X_val = np.array(X[train_cut_index:val_cut_index,:])
X_test = np.array(X[val_cut_index:last_test_index,:])

y_train = y[0:(train_cut_index)]
y_val = y[train_cut_index:val_cut_index]
y_test = y[val_cut_index:last_test_index]

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
#Preproc the y_train and y_test using to_categorical function
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [ ]:
#Reshape data for inputting into LSTM
X_train = np.reshape(X_train, (int(X_train.shape[0]/timestep), timestep, X_train.shape[1]))
X_val = np.reshape(X_val, (int(X_val.shape[0]/timestep), timestep, X_val.shape[1]))
X_test = np.reshape(X_test, (int(X_test.shape[0]/timestep), timestep, X_test.shape[1]))

y_train = np.reshape(y_train, (int(y_train.shape[0]/timestep), timestep, 2))
y_val = np.reshape(y_val, (int(y_val.shape[0]/timestep), timestep, 2))
y_test = np.reshape(y_test, (int(y_test.shape[0]/timestep), timestep, 2))

In [ ]:
X.iloc[0]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
#y_train

## Step 4: Fit model


In [ ]:
#Import Library
from keras.layers import Activation, Dense, LSTM, Dropout

In [ ]:
#Define Parameters
learning_rate = 0.001
epochs= 100
loss = 'categorical_crossentropy'
dropout=0.5

In [ ]:
#Optimizer
optimizer = optimizers.Adam(lr=learning_rate)

In [ ]:
#Model Architect
#-----------------

model = Sequential()

#Stack LSTM Cells
model.add(LSTM(100, input_shape=X_train.shape[1:], batch_size=1, return_sequences=True, stateful=True))

model.add(LSTM(50, batch_size=1, return_sequences=True, stateful=True))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(2, activation='softmax'))


model.summary()

In [ ]:
#Compile
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
#Fit
fitted = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=1, verbose=1, shuffle=False)

## Step 5: Evaluate


In [ ]:
### Plot Training vs Validation Curve
plt.figure()
plt.plot(fitted.history['loss'])
plt.plot(fitted.history['val_loss'])
plt.title('Training Loss & Validation Loss')
plt.ylabel('Binary Cross Entropy Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()

In [ ]:
### Plot Training vs Validation Accuracy
plt.figure()
plt.plot(fitted.history['acc'])
plt.plot(fitted.history['val_acc'])
plt.title('Training Accuracy & Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.show()